In [83]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm import tqdm_notebook, tqdm

In [2]:
def load_data(filename):
    data = pd.read_csv(filename,sep=' ').reset_index()
    data = data.rename(columns={data.columns[0]:'x', data.columns[1]:'y'})
    return data

In [3]:
def dist2(p1,p2):
    return (p1['x']-p2['x'])**2 + (p1['y']-p2['y'])**2

In [4]:
def dist(p1,p2):
    return np.sqrt(dist2(p1,p2))

In [5]:
def closest_point(visited_point, data):
    p0 = data.loc[visited_point[-1]]
    return data.loc[~data.index.isin(visited_point)].apply(
        lambda p: dist2(p,p0),
        axis=1
    ).argmin()

In [6]:
def tsp_simple(data):
    sol = [data.index[0]]
    while len(sol) < len(data):
        sol += [closest_point(sol,data)]
    return sol

In [7]:
def tsp_simple_fast(data,closest_matrix):
    sol = [data.index[0]]
    remainings = set(range(len(data)))
    while len(sol) < len(data):
        last = sol[-1]
        if len(closest_matrix[last][~closest_matrix[last].isin(sol)])>0:
            sol += [closest_matrix[last][~closest_matrix[last].isin(sol)].values[0]]
        else:
            sol += [np.random.choice(list(remainings))] 
        remainings -= set([sol[-1]])
    return sol

In [8]:
def objective(sol, data):
    return sum([dist(data.loc[p1],data.loc[p2]) for p1, p2 in zip(sol,sol[1:]+[sol[0]])])  

In [9]:
def objective2_vec(sol, data):
    return [dist2(data.loc[p1],data.loc[p2]) for p1, p2 in zip(sol,sol[1:]+[sol[0]])] 

In [79]:
def objective2_vec_fast(sol, data):
    return map(lambda x: x[0]**2+x[1]**2,data.loc[sol].values-data.loc[shift(sol,1)].values)

In [80]:
def objective_fast(sol, data):
    return np.sqrt(sum(objective2_vec_fast(sol, data)))  

In [11]:
def save_sol(sol, filename):
    data = load_data(filename)
    obj = objective(sol,data)
    output_data = str(obj) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, sol))
    f = open(filename.replace('data','out'),'w')
    f.write(output_data)
    f.close()

In [12]:
def shift(seq, n):
    return seq[n:]+seq[:n]

In [13]:
def swap_tsp_2(sol, p1, p2):
    pos1 = sol.index(p1)
    shift_sol = shift(sol,pos1)
    pos2 = shift_sol.index(p2)
    res_sol = shift_sol[:pos2]+shift_sol[pos2:][::-1]
    return shift(res_sol,len(sol)-pos1)

In [14]:
def objective_swap(sol, obj, p1 ,p2, data):
    pos1 = sol.index(p1)
    pos2 = sol.index(p2)
    p1_next = sol[pos1-1]
    p2_next = sol[pos2-1]
    dp1 = data.loc[p1]
    dp2 = data.loc[p2]
    dp1_next = data.loc[p1_next]
    dp2_next = data.loc[p2_next]
    return obj - dist2(dp1,dp1_next) - dist2(dp2,dp2_next) + dist2(dp1,dp2) + dist2(dp1_next,dp2_next)

In [15]:
def objective_vec_swap(sol, obj_vec, p1 ,p2, data):
    pos1 = sol.index(p1)
    shift_sol = shift(sol,pos1)
    shift_obj_vec = shift(obj_vec,pos1)
    pos2 = shift_sol.index(p2)
    res_obj_vec = shift_obj_vec[:pos2]+ (shift_obj_vec[pos2:][::-1][1:]+[0])
    p1_next = sol[pos1-1]
    p2_next = shift_sol[pos2-1]
    dp1 = data.loc[p1]
    dp2 = data.loc[p2]
    dp1_next = data.loc[p1_next]
    dp2_next = data.loc[p2_next]
    res_obj_vec[pos2-1]=dist2(dp1_next,dp2_next)
    res_obj_vec[-1]=dist2(dp1,dp2)
    return shift(res_obj_vec,len(sol)-pos1)

In [16]:
from collections import deque

In [17]:
def local_search(
    data,
    max_trials,
    start_sol,
    start_obj_vec,
    closest_matrix,
    first_closest=None,
    tabu_flag = False,
    tabu_frac=1.0,
    tabu_size=1000):

    size = len(start_sol)
    tabu_size = int(tabu_frac*size)
    cur_sol = start_sol
    cur_obj_vec = start_obj_vec
    cur_obj = sum(cur_obj_vec)
    opt_sol = start_sol
    opt_obj_vec = cur_obj_vec
    opt_obj = cur_obj

    tabu_list = deque([])
    it = 0
    print 'it 0 obj: %f'%(sum(map(np.sqrt,cur_obj_vec)))
    while (it < max_trials):
        order_points = pd.Series(cur_obj_vec).sort_values(ascending=False).index
        max_trials_reached = False
        local_opt_sol = None
        local_opt_obj_vec = None
        local_opt_obj = None
        for p1 in order_points[:first_closest]:
            for idx2 in closest_matrix[p1].index[1:first_closest if first_closest is not None else size]:
                if it >= max_trials:
                    max_trials_reached = True
                    break
                else:
                    it+=1
                p2 = closest_matrix[p1][idx2]
                #print 'try (%d %d)'%(p1,p2)
                new_obj = objective_swap(cur_sol, cur_obj, p1, p2, data)         
                if (local_opt_sol is None) or (new_obj < local_opt_obj):
                    tmp_obj_vec = objective_vec_swap(cur_sol,cur_obj_vec,p1,p2,data) 
                    tabu_check = pd.Series(tmp_obj_vec).sort_values(ascending=False).values[:tabu_size].tolist()
                    if (tabu_flag and tabu_check not in tabu_list)  or \
                    not tabu_flag:
                        if local_opt_sol is None:
                            print 'it %d init order %d closest rank %d swap (%d %d) obj: %f'%(it, order_points.get_loc(p1),idx2, p1,p2,sum(map(np.sqrt,tmp_obj_vec)))
                        else:
                            print 'it %d order %d closest rank %d swap (%d %d) obj: %f'%(it, order_points.get_loc(p1),idx2, p1,p2,sum(map(np.sqrt,tmp_obj_vec)))
                        local_opt_sol = swap_tsp_2(cur_sol,p1,p2)
                        local_opt_obj_vec = tmp_obj_vec
                        local_opt_obj = new_obj
                        tabu_list.appendleft([pd.Series(local_opt_obj_vec).sort_values(ascending=False).values[:tabu_size].tolist()])
            if max_trials_reached:
                break
        while (len(tabu_list)>tabu_size):
            #print 'empty tabu list'
            tabu_list.pop()
        cur_sol = local_opt_sol
        cur_obj_vec = local_opt_obj_vec
        cur_obj = local_opt_obj
        if local_opt_obj < opt_obj:
            opt_sol[:] = local_opt_sol
            opt_obj_vec[:] = local_opt_obj_vec
            opt_obj = local_opt_obj
            print 'it %d obj: %f'%(it,sum(map(np.sqrt,opt_obj_vec)))
    return sum(map(np.sqrt,opt_obj_vec)), opt_sol

In [18]:
from datetime import datetime

In [19]:
def dist_matrix_calc(data):
    data_x = np.repeat(data['x'].values,len(data)).reshape((len(data),len(data)))
    data_y = np.repeat(data['y'].values,len(data)).reshape((len(data),len(data)))
    return pd.DataFrame((data_x-data_x.T)*(data_x-data_x.T)+(data_y-data_y.T)*(data_y-data_y.T))

In [20]:
def closest_matrix_calc_step(data, start_idx, end_idx, first_closest):
    data_x = data.x.values
    data_y = data.y.values
    data_x_1 = np.repeat(data_x,end_idx-start_idx).reshape(len(data),end_idx-start_idx)
    data_x_2 = np.repeat(data_x[start_idx:end_idx],len(data)).reshape((end_idx-start_idx,len(data))).T
    data_y_1 = np.repeat(data_y,end_idx-start_idx).reshape(len(data),end_idx-start_idx)
    data_y_2 = np.repeat(data_y[start_idx:end_idx],len(data)).reshape((end_idx-start_idx,len(data))).T
    dist_matrix = pd.DataFrame((data_x_2-data_x_1)*(data_x_2-data_x_1)
                               +(data_y_2-data_y_1)*(data_y_2-data_y_1),
                              columns=range(start_idx,end_idx))
    return pd.DataFrame([dist_matrix[c].nsmallest(first_closest).index for c in dist_matrix.columns],
                        index=range(start_idx, end_idx)).T

In [21]:
def closest_matrix_calc(data, first_closest, step=100):
    steps = range(0,len(data),step)+[len(data)]
    return pd.concat([closest_matrix_calc_step(data,start_idx,end_idx,first_closest) 
               for (start_idx, end_idx) in zip(steps[:-1],steps[1:])],axis=1)

In [22]:
def solve_tsp(filename,max_trials,
              first_closest,
              tabu_flag = False,
              tabu_frac = 1.0,
              tabu_size= 1000,
              closest_matrix=None,
              calc_first_closest=1000,
              simple_fast_flag = True):
    data = load_data(filename)
    start_time = datetime.now()
    if closest_matrix is None:
        closest_matrix = closest_matrix_calc_kd(data,calc_first_closest)
    closest_matrix_time = datetime.now()
    print 'closest_matrix time : %d'%((closest_matrix_time-start_time).seconds)
    if simple_fast_flag:
        sol_step0 = tsp_simple_fast(data,closest_matrix)
    else:
        sol_step0 = tsp_simple(data)
    start_sol_time = datetime.now()
    print 'start sol time : %d'%((start_sol_time-closest_matrix_time).seconds)
    obj_vec = objective2_vec(sol_step0,data)
    opt_obj, opt_sol = local_search(data,max_trials,sol_step0,obj_vec,closest_matrix,first_closest,
                                    tabu_flag,
                                    tabu_frac,
                                    tabu_size)
    start_local_search_time = datetime.now()
    print 'local_search time : %d'%((start_local_search_time-start_sol_time).seconds)
    return opt_obj, opt_sol

In [23]:
PbFilename1 = r'./data/tsp_51_1'
PbFilename2 = r'./data/tsp_100_3'
PbFilename3 = r'./data/tsp_200_2'
PbFilename4 = r'./data/tsp_574_1'
PbFilename5 = r'./data/tsp_1889_1'
PbFilename6 = r'./data/tsp_33810_1'

In [24]:
def closest_matrix_calc_kd(data,first_closest):
    kd = KDTree(data.values)
    return pd.DataFrame([kd.query(data.loc[idx].values,first_closest)[1] for idx in tqdm_notebook(xrange(len(data)), 
                                                                                                  desc='closest matrix')]).T

In [85]:
def tsp_simple_fast_kd(data, step=30):
    sol = [data.index[0]]
    kd = KDTree(data.values)
    remainings = set(range(len(data)))
    bar = tqdm_notebook(total=len(data))
    idx = 0
    while len(sol) < len(data):
        last = sol[-1]
        not_found = True 
        while not_found:
            closest = kd.query(data.loc[last].values,step)[1]
            closest_remaining = list(set(closest)&remainings) 
            if len(closest_remaining) > 0:
                not_found = False
            else:
                kd = KDTree(data.loc[list(remainings)].values)
                
        bar.update(1)
        idx = idx + 1
        sol += [closest_remaining[0]]
    
        remainings -= set([sol[-1]])
    return sol

In [95]:
import solver
reload(solver)

<module 'solver' from 'solver.py'>

In [97]:
data = load_data(PbFilename4)
first_sol = solver.tsp_simple_fast_kd(data,10)

HBox(children=(IntProgress(value=0, max=573), HTML(value=u'')))

In [28]:
#%time obj2_vec = objective2_vec(fist_sol,data)

In [50]:
%matplotlib notebook

In [51]:
import matplotlib.pyplot as plt

In [98]:
def plot_sol(data,sol):
    plt.plot(data.loc[sol].x,data.loc[sol].y)
    plt.scatter(data.loc[sol].x,data.loc[sol].y,color='r',s=5)

In [40]:
clusters = kmeans(data.values.astype(float),100)

In [76]:
pd.CategoricalIndex(data.index).shift(1)

NotImplementedError: Not supported for type CategoricalIndex

In [34]:
np.sqrt(sum(obj2_vec))

1187619.6181858904

In [55]:
%time cm = closest_matrix_calc_kd(data, 1000)

Wall time: 1d 8h 32min 27s


In [49]:
save_sol(opt_sol,PbFilename6)

In [50]:
len(opt_sol)

33810

In [48]:
opt_obj, opt_sol = solve_tsp(PbFilename6,
                             50000,
                             20,
                             calc_first_closest=20,
                             step=1000,
                             tabu_flag=True,
                             tabu_frac=0.1,
                             tabu_size = 1000,       
                             simple_fast_flag=True)
print opt_obj

closest_matrix time : 617
start sol time : 355
it 0 obj: 320109014.104850
it 1 init order 0 closest rank 1 swap (32336 31219) obj: 320109014.104842
it 98 order 5 closest rank 3 swap (32756 32755) obj: 320109368.490564
it 252 order 13 closest rank 5 swap (33545 33534) obj: 320108330.963316
it 355 order 18 closest rank 13 swap (32191 32212) obj: 320108730.814855
it 380 obj: 320108730.814855
it 381 init order 0 closest rank 1 swap (32336 31219) obj: 320108730.814855
it 420 order 2 closest rank 2 swap (18780 18781) obj: 320105555.016199
it 427 order 2 closest rank 9 swap (18780 18766) obj: 320106394.155418
it 466 order 4 closest rank 10 swap (18404 18406) obj: 320107509.910817
it 614 order 12 closest rank 6 swap (8250 8257) obj: 320105116.850392
it 760 obj: 320105116.850392
it 761 init order 0 closest rank 1 swap (32336 31219) obj: 320105116.850387
it 932 order 9 closest rank 1 swap (21929 21920) obj: 320102373.426336
it 937 order 9 closest rank 6 swap (21929 21911) obj: 320102659.373709
i

it 8764 order 1 closest rank 5 swap (20897 20271) obj: 319668005.420923
it 8887 order 7 closest rank 14 swap (482 473) obj: 319667228.549948
it 9120 obj: 319667228.549948
it 9121 init order 0 closest rank 1 swap (17096 17095) obj: 319667228.549916
it 9144 order 1 closest rank 5 swap (20897 20271) obj: 319664829.917622
it 9500 obj: 319664829.917622
it 9501 init order 0 closest rank 1 swap (24777 23680) obj: 319664829.917622
it 9504 order 0 closest rank 4 swap (24777 24782) obj: 319664103.810173
it 9646 order 7 closest rank 13 swap (7581 6189) obj: 319664991.732057
it 9849 order 18 closest rank 7 swap (7574 7572) obj: 319662762.002865
it 9880 obj: 319662762.002865
it 9881 init order 0 closest rank 1 swap (17332 17331) obj: 319662762.002865
it 9894 order 0 closest rank 14 swap (17332 17334) obj: 319664076.145725
it 10260 obj: 319664076.145725
it 10261 init order 0 closest rank 1 swap (17332 17331) obj: 319662762.002865
it 10262 order 0 closest rank 2 swap (17332 17330) obj: 319664076.1456

it 18138 order 14 closest rank 12 swap (16291 15820) obj: 319304914.821737
it 18240 obj: 319304914.821737
it 18241 init order 0 closest rank 1 swap (16331 15849) obj: 319304914.821737
it 18262 order 1 closest rank 3 swap (27369 27370) obj: 319303386.957692
it 18303 order 3 closest rank 6 swap (26104 26094) obj: 319305566.314716
it 18473 order 12 closest rank 5 swap (25372 26259) obj: 319306278.777114
it 18540 order 15 closest rank 15 swap (4579 4571) obj: 319301770.070676
it 18588 order 18 closest rank 6 swap (22325 22333) obj: 319302493.289222
it 18590 order 18 closest rank 8 swap (22325 22323) obj: 319304463.183585
it 18594 order 18 closest rank 12 swap (22325 21528) obj: 319304322.777433
it 18599 order 18 closest rank 17 swap (22325 22332) obj: 319304237.998313
it 18620 obj: 319304237.998313
it 18621 init order 0 closest rank 1 swap (16331 15849) obj: 319304237.998327
it 18622 order 0 closest rank 2 swap (16331 15850) obj: 319301006.790289
it 18952 order 17 closest rank 9 swap (2643

it 28120 obj: 319269852.178338
it 28121 init order 0 closest rank 1 swap (31085 31078) obj: 319269852.178338
it 28187 order 3 closest rank 10 swap (19531 19520) obj: 319270572.385252
it 28293 order 9 closest rank 2 swap (28417 28416) obj: 319267542.850963
it 28500 obj: 319267542.850963
it 28501 init order 0 closest rank 1 swap (613 603) obj: 319267542.850963
it 28567 order 3 closest rank 10 swap (19531 19520) obj: 319269207.136007
it 28570 order 3 closest rank 13 swap (19531 19550) obj: 319266703.040492
it 28880 obj: 319266703.040492
it 28881 init order 0 closest rank 1 swap (613 603) obj: 319266703.040492
it 28940 order 3 closest rank 3 swap (19531 19532) obj: 319264230.904537
it 29104 order 11 closest rank 15 swap (17486 32280) obj: 319273027.297202
it 29122 order 12 closest rank 14 swap (8912 8926) obj: 319268393.267505
it 29215 order 17 closest rank 12 swap (3316 1540) obj: 319267626.921277
it 29237 order 18 closest rank 15 swap (13475 15674) obj: 319264641.202538
it 29260 obj: 319

it 37242 order 0 closest rank 2 swap (4593 4594) obj: 318834759.964660
it 37244 order 0 closest rank 4 swap (4593 4584) obj: 318833944.682954
it 37277 order 1 closest rank 18 swap (17979 16917) obj: 318840540.481678
it 37620 obj: 318840540.481678
it 37621 init order 0 closest rank 1 swap (5173 5166) obj: 318840540.481669
it 37642 order 1 closest rank 3 swap (17979 17977) obj: 318842154.857092
it 37720 order 5 closest rank 5 swap (18859 18871) obj: 318839725.199972
it 37875 order 13 closest rank 8 swap (26283 26294) obj: 318840514.958181
it 37904 order 14 closest rank 18 swap (5133 5154) obj: 318840406.597116
it 38000 obj: 318840406.597116
it 38001 init order 0 closest rank 1 swap (6102 6101) obj: 318840406.597119
it 38045 order 2 closest rank 7 swap (12393 12402) obj: 318839591.315412
it 38063 order 3 closest rank 6 swap (1799 1791) obj: 318841191.701550
it 38093 order 4 closest rank 17 swap (24683 23677) obj: 318841741.117120
it 38310 order 16 closest rank 6 swap (16292 16295) obj: 31

it 44460 obj: 318269640.421283
it 44461 init order 0 closest rank 1 swap (30038 31195) obj: 318269640.421283
it 44523 order 3 closest rank 6 swap (25641 27738) obj: 318269430.057537
it 44538 order 4 closest rank 2 swap (12331 12330) obj: 318268119.018576
it 44539 order 4 closest rank 3 swap (12331 12340) obj: 318267689.914002
it 44544 order 4 closest rank 8 swap (12331 11789) obj: 318266141.534620
it 44549 order 4 closest rank 13 swap (12331 12338) obj: 318266884.791238
it 44553 order 4 closest rank 17 swap (12331 12348) obj: 318266220.008256
it 44840 obj: 318266220.008256
it 44841 init order 0 closest rank 1 swap (30038 31195) obj: 318266220.008243
it 44903 order 3 closest rank 6 swap (25641 27738) obj: 318266184.532762
it 45004 order 8 closest rank 12 swap (12515 12506) obj: 318266020.806458
it 45220 obj: 318266020.806458
it 45221 init order 0 closest rank 1 swap (30038 31195) obj: 318266020.806445
it 45283 order 3 closest rank 6 swap (25641 27738) obj: 318265985.330964
it 45472 orde

In [58]:
set(range(33810)) -set(opt_sol)

{3548}

In [54]:
len(data)

NameError: name 'data' is not defined